In [45]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import os
from tqdm.auto import tqdm

In [47]:
import numpy as np
import pandas as pd
from pm4py.objects.log.util import dataframe_utils
import time
import joblib

In [48]:
os.listdir()

['.ipynb_checkpoints',
 '01_Get_config-semidone.ipynb',
 '02_Comp_load-combined-done.ipynb',
 'configs',
 'dur_range_dic.pickle',
 'evlog.csv',
 'get_dur_range_dic.ipynb',
 'get_load_locations_done.ipynb',
 'load_locations.pickle',
 'old_scripts']

In [49]:
def get_lead_ts(log):
    
    temp_log = log.copy()
    temp_log = temp_log.sort_values(['case_id', 'event_id'])
    temp_log['ts_next'] = temp_log.ts.shift(-1)
    temp_log.loc[temp_log['activity'] == '<EOS>', 'ts_next'] = np.nan
    
    return log.merge(temp_log[['event_id', 'ts_next']], left_on='event_id', right_on='event_id')
  

def get_all_configs(conf_dir = 'configs/'):
        
    configurations = {}

    for conf in os.listdir(conf_dir):

        if 'configuration' in conf:
            configurations.update(joblib.load(conf_dir + conf))

    print('config keys:', list(configurations.keys()))
    
    return configurations

def fill_na_config(configs, na_val = 60*24):
    
    for load_activity in list(configs.keys()):
        #print(load_activity)
        for key, val in zip(configs['{}'.format(load_activity)].keys(), configs['{}'.format(load_activity)].values()):

            if pd.isna(val[0]):
                configs['{}'.format(load_activity)][key] = (configs['{}'.format(load_activity)][key][0], configs['{}'.format(load_activity)][key][1], (na_val))
                #print(key, configs['{}'.format(load_activity)][key])
                
    return configs

    

In [54]:
def load_state_optdur(x, previous, load, configurations):
    offset = pd.DateOffset(minutes=0)
    diff = pd.DateOffset(minutes=configurations[load][x['activity']][2])
    return previous.loc[(previous >= x['ts']-diff-offset) & (previous < x['ts']-offset)].count()

def load_state_activecases(x, previous):
    
    return previous.loc[(previous.ts <= x.ts) & (previous.ts_next >= x.ts)].ts.count()    
   
    
def get_load_log(log, load_log=None, load_state = 'optdur'):
    """returns a log with loads, if load_log is not provided loads are added to all events in log. Both load_state=optdur or actcase are supported."""

    if load_state == 'actcase':
        log = get_lead_ts(log)
    
    elif load_state == 'optdur':
        configurations = get_all_configs()
        configurations = fill_na_config(configurations)
        
    else:
        print(f'load state: {load_state}, not supported')
        return None
    
    locations = joblib.load('load_locations.pickle')
    
    if load_log is None:
        load_log = log.copy()
        print('Im here')

    for location in tqdm(locations):
        
        if load_state =='actcase':
            previous = log.loc[log.activity == location][['ts', 'ts_next']]
            load_comp = pd.DataFrame(load_log.apply(lambda x: load_state_activecases(x, previous), axis=1)) #this is causing the warning msg
        
        elif load_state == 'optdur':
            previous = log.loc[log.activity == location, 'ts']
            load_comp = pd.DataFrame(load_log.apply(lambda x: load_state_optdur(x, previous, location, configurations), axis=1)) #this is causing the warning msg

        load_log['load_{}'.format(location)] = load_comp
        
    return load_log
    

In [55]:
log_csv = pd.read_csv('evlog.csv', sep=',')
#log_csv.ts = log_csv.ts.apply(lambda x: x[:-4])
log_csv.drop(log_csv.columns[0], axis=1, inplace=True)
log_csv = dataframe_utils.convert_timestamp_columns_in_df(log_csv)
log_csv = log_csv.sort_values('ts')
log_csv.set_index(log_csv.ts, inplace=True)


In [56]:
load_log = log_csv[log_csv.case_id.isin(list(np.random.choice(log_csv.case_id.unique(), size=10, replace=False )))]

In [59]:
load_log = get_load_log(log=log_csv, load_log=load_log, load_state='actcase')

D:\Users\u0126338\Anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Users\u0126338\Anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Users\u0126338\Anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

D:\Users\u0126338\Anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [60]:
load_log

,event_id,case_id,activity,ts,duration,remtime,load_W_Completeren aanvraag,load_W_Nabellen offertes,load_W_Nabellen incomplete dossiers,load_W_Valideren aanvraag,load_W_Afhandelen leads
ts,,,,,,,,,,,
2011-11-02 15:47:05.468000+00:00,58827,181679,<BOS>,2011-11-02 15:47:05.468000+00:00,0.0,1035940.0,228,397,45,8,3
2011-11-02 15:47:05.468000+00:00,58828,181679,A_SUBMITTED,2011-11-02 15:47:05.468000+00:00,0.0,1035940.0,228,397,45,8,3
2011-11-02 15:47:05.686000+00:00,58829,181679,A_PARTLYSUBMITTED,2011-11-02 15:47:05.686000+00:00,1.0,1035939.0,228,397,45,8,3
2011-11-02 15:47:37.293000+00:00,58830,181679,A_PREACCEPTED,2011-11-02 15:47:37.293000+00:00,32.0,1035908.0,228,397,45,8,3
2011-11-02 15:47:37.849000+00:00,58831,181679,W_Completeren aanvraag,2011-11-02 15:47:37.849000+00:00,33.0,1035907.0,229,397,45,8,3
...,...,...,...,...,...,...,...,...,...,...,...
2012-02-29 21:51:12.326000+00:00,288144,214328,<BOS>,2012-02-29 21:51:12.326000+00:00,0.0,40.0,286,504,76,10,14
2012-02-29 21:51:12.326000+00:00,288145,214328,A_SUBMITTED,2012-02-29 21:51:12.326000+00:00,0.0,40.0,286,504,76,10,14
2012-02-29 21:51:12.501000+00:00,288146,214328,A_PARTLYSUBMITTED,2012-02-29 21:51:12.501000+00:00,1.0,39.0,286,504,76,10,14
